# Introduction to getObsAtmo

- atuhor : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2025/10/17
- last update : 2025/10/17 : getObsAtmo version 0.2.2

**Goal** : Top page to access to notebooks

## Notebook demonstrating the use of getObsAtmo

- [demo_notebook.ipynb](demo_notebook.ipynb)

## Notebook showing the altitude of the different observatory sites defined in getObsAtmo

- This notebook illustrates the use of the atmospheric analytical package `atmanalyticalmodels`

- [AtmosphericPressureDensityProfiles.ipynb](AtmosphericPressureDensityProfiles.ipynb)

## Notebook for comparing Rayleigh scattering with analytical model

- This notebook illustrates the use of the atmospheric analytical package `atmanalyticalmodels`

- [compareRayleighScattAnalyticalModel_notebook.ipynb](compareRayleighScattAnalyticalModel_notebook.ipynb)